# <center> Home Credit - Credit Risk Model Stability Data Challenge</center>

---

Link to the challenge: https://www.kaggle.com/competitions/home-credit-credit-risk-model-stability/data



In [57]:
import pandas as pd
import numpy as np

import plotly.express as px

from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from scipy import stats


In [2]:
##### Functions used in notebook

def dataframe_summary(dataframe: pd.DataFrame,
                      max_length: int):

    ### Get list of column names
    col_names = dataframe.columns.to_list()

    ### Get unique count of each column
    col_length = [len(dataframe[val].unique()) for val in col_names]

    ### Append info for dataframe
    col_names.append("this_df")
    col_length.append(len(dataframe))

    ### Create new dataframe for summary
    summary_df = pd.DataFrame(list(zip(col_names, col_length)), columns=["column_name", "unique_count"])

    ### Count nan values in each column
    nan_df = dataframe.isnull().sum(axis = 0).to_frame().reset_index().rename(columns={"index":"column_name", 0:"nan_count"})
    nan_df["nan_percent"] = nan_df["nan_count"]/len(dataframe)*100

    ### Create new frame for data type in each column
    type_df = dataframe.dtypes.to_frame().reset_index().rename(columns={"index":"column_name", 0:"data_type"})

    ### Create new dataframe with unique entries for each column

    col_name = []
    unique_val = []
    
    for val  in dataframe.columns.to_list():

        col_name.append(val)

        if len(dataframe[val].unique())<=max_length:
            unique_val.append(list(dataframe[val].unique()))

        else:
            unique_val.append(f">{max_length} entries")

    unique_df = pd.DataFrame(list(zip(col_name, unique_val)),
                             columns=["column_name", "unique_entries"])
    

    ### Merge summary_df with nan_df, type_df
    summary_df = summary_df.merge(nan_df, on="column_name", how="outer")
    summary_df = summary_df.merge(type_df, on="column_name", how="outer")
    summary_df = summary_df.merge(unique_df, on="column_name", how="outer")


    return summary_df.sort_values(["data_type", "nan_count"])


def describe_dataframe(dataframe: pd.DataFrame):
    
    pd.options.display.float_format = '{:,.3f}'.format
    
    return dataframe.select_dtypes([int, float]).describe()


---
---
# <center> Data Exploration </center>
---



In [3]:
base_df = pd.read_csv("./csv_files/train/train_base.csv")
base_df.head()

,case_id,date_decision,MONTH,WEEK_NUM,target
0,0,2019-01-03,201901,0,0
1,1,2019-01-03,201901,0,0
2,2,2019-01-04,201901,0,0
3,3,2019-01-03,201901,0,0
4,4,2019-01-04,201901,0,1


In [4]:
base_summary = dataframe_summary(base_df, 10)
base_summary

,column_name,unique_count,nan_count,nan_percent,data_type,unique_entries
0,case_id,1526659,0.0,0.0,int64,>10 entries
2,MONTH,22,0.0,0.0,int64,>10 entries
3,WEEK_NUM,92,0.0,0.0,int64,>10 entries
4,target,2,0.0,0.0,int64,"[0, 1]"
1,date_decision,644,0.0,0.0,object,>10 entries
5,this_df,1526659,NaN,NaN,NaN,NaN


In [6]:
base_df.groupby("target").count()

,case_id,date_decision,MONTH,WEEK_NUM
target,,,,
0,1478665,1478665,1478665,1478665
1,47994,47994,47994,47994


In [8]:
47994/1478665*100

3.2457656061379687

Notes on base_df:
* MONTH and WEEK_NUM will only be used if there's a need for aggregation using date
* `case_id` will be used to join tables
* The 'positive' or 'true' cases is roughly 3.25% only. This is a very unbalanced dataset. Need to figure out how to deal with this.

In [5]:
bureau_b_1 = pd.read_csv("./csv_files/train/train_credit_bureau_b_1.csv")

##### Cast column as datetime type
bureau_b_1["contractdate_551D"] = pd.to_datetime(bureau_b_1["contractdate_551D"])
bureau_b_1["contractmaturitydate_151D"] = pd.to_datetime(bureau_b_1["contractmaturitydate_151D"])

##### Derive columns
bureau_b_1["contract_length_year"] = (bureau_b_1["contractmaturitydate_151D"] - bureau_b_1["contractdate_551D"]).dt.days/365

bureau_b_1.head(5)

/var/folders/pq/7mcbx8c97t90g8y2_ns4gjkr0000gn/T/ipykernel_13674/601919751.py:1: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  bureau_b_1 = pd.read_csv("./csv_files/train/train_credit_bureau_b_1.csv")


,case_id,amount_1115A,classificationofcontr_1114M,contractdate_551D,contractmaturitydate_151D,contractst_516M,contracttype_653M,credlmt_1052A,credlmt_228A,credlmt_3940954A,...,pmtnumpending_403L,purposeofcred_722M,residualamount_1093A,residualamount_127A,residualamount_3940956A,subjectrole_326M,subjectrole_43M,totalamount_503A,totalamount_881A,contract_length_year
0,467,NaN,ea6782cc,2011-06-15,2031-06-13,7241344e,724be82a,3000000.0,10000.0,3000000.0,...,NaN,96a8fdfe,0.0,0.0,NaN,fa4f56f1,ab3c25cf,3000000.00,10000.0,20.008219
1,467,NaN,ea6782cc,2019-01-04,2021-08-04,7241344e,724be82a,NaN,NaN,130365.0,...,NaN,96a8fdfe,NaN,NaN,NaN,ab3c25cf,ab3c25cf,78000.00,960000.0,2.583562
2,467,78000.0,ea6782cc,2016-10-25,2019-10-25,7241344e,4257cbed,NaN,NaN,NaN,...,10.0,96a8fdfe,NaN,NaN,NaN,a55475b1,a55475b1,NaN,NaN,3.000000
3,1445,NaN,ea6782cc,2015-01-30,2021-01-30,7241344e,1c9c5356,400000.0,100000.0,74000.0,...,NaN,60c73645,0.0,0.0,73044.18,daf49a8a,ab3c25cf,400000.00,100000.0,6.005479
4,1445,NaN,01f63ac8,2014-09-12,2021-09-12,7241344e,724be82a,NaN,NaN,400000.0,...,NaN,96a8fdfe,NaN,NaN,NaN,ab3c25cf,ab3c25cf,396800.62,184587.8,7.005479


In [34]:
bureau_b_1_summary = dataframe_summary(bureau_b_1, 15)
bureau_b_1_summary

,column_name,unique_count,nan_count,nan_percent,data_type,unique_entries
0,case_id,36500,0.0,0.000000,int64,>15 entries
27,num_group1,21,0.0,0.000000,int64,>15 entries
45,contract_length_year,2669,4079.0,4.754578,float64,>15 entries
17,dpdmax_851P,20158,4567.0,5.323402,float64,>15 entries
18,dpdmaxdatemonth_804T,13,4567.0,5.323402,float64,"[nan, 11.0, 1.0, 8.0, 6.0, 3.0, 7.0, 12.0, 10...."
19,dpdmaxdateyear_742T,16,4567.0,5.323402,float64,>15 entries
26,maxdebtpduevalodued_3940955A,1474,4567.0,5.323402,float64,>15 entries
29,overdueamountmax_950A,1835,4567.0,5.323402,float64,>15 entries
30,overdueamountmaxdatemonth_494T,13,4567.0,5.323402,float64,"[nan, 11.0, 2.0, 1.0, 8.0, 6.0, 12.0, 10.0, 9...."
31,overdueamountmaxdateyear_432T,16,4567.0,5.323402,float64,>15 entries


In [35]:
bureau_b_1_desc = describe_dataframe(bureau_b_1)
bureau_b_1_desc

,case_id,amount_1115A,credlmt_1052A,credlmt_228A,credlmt_3940954A,credquantity_1099L,credquantity_984L,debtpastduevalue_732A,debtvalue_227A,dpd_550P,...,overdueamountmaxdatemonth_494T,overdueamountmaxdateyear_432T,pmtdaysoverdue_1135P,pmtnumpending_403L,residualamount_1093A,residualamount_127A,residualamount_3940956A,totalamount_503A,totalamount_881A,contract_length_year
count,"85,791.000","43,681.000","27,581.000","16,130.000","38,218.000","53,018.000","46,228.000","81,217.000","43,681.000","53,018.000",...,"81,224.000","81,224.000","81,217.000","43,680.000","16,125.000","27,581.000","37,519.000","53,018.000","46,228.000","81,712.000"
mean,"1,218,998.116","214,110.489","178,935.650","52,317.523","130,360.344",1.545,4.565,"3,791.878","165,118.331","25,696.752",...,6.631,"2,018.291",71.479,20.265,0.020,"57,942.299","43,011.067","257,008.919","293,763.164",3.271
std,"686,332.016","691,019.602","5,274,021.553","128,082.458","2,570,305.488",0.864,5.170,"158,238.060","550,497.133","968,643.445",...,3.495,1.741,"4,664.511",29.017,2.536,"110,621.466","93,146.195","3,876,513.325","1,202,831.312",2.787
min,467.000,0.200,0.000,0.000,0.000,1.000,1.000,0.000,0.000,0.000,...,1.000,"1,900.000",0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,"727,201.000","25,998.000",0.000,0.000,0.000,1.000,1.000,0.000,"12,946.000",0.000,...,4.000,"2,018.000",0.000,5.000,0.000,0.000,0.000,"20,000.000","25,721.700",1.504
50%,"1,413,976.000","60,000.000","36,184.000","22,600.000","20,000.000",1.000,3.000,0.000,"35,893.414",0.000,...,7.000,"2,019.000",0.000,11.000,0.000,"14,461.327","7,483.786","68,379.600","84,441.003",2.003
75%,"1,778,253.000","160,000.000","121,778.000","60,000.000","78,000.000",2.000,6.000,0.000,"110,801.625",0.000,...,10.000,"2,019.000",0.000,24.000,0.000,"70,925.400","41,320.617","200,000.000","276,102.200",4.003
max,"2,703,436.000","54,833,332.000","796,800,000.000","4,420,000.000","300,000,000.000",16.000,146.000,"41,138,710.000","41,619,050.000","207,823,776.000",...,12.000,"2,020.000","663,618.000",300.000,322.000,"2,187,568.200","2,022,909.200","796,800,000.000","139,080,000.000",31.022


In [ ]:
numerical_bureau = bureau_b_1.select_dtypes([int, float])

bureau_corr_mat = numerical_bureau.corr()

bureau_corr_mat

In [16]:
fig_bureau_a = px.imshow(bureau_corr_mat)

fig_bureau_a.update_layout(
    height=1200,
    width=1200
)
fig_bureau_a.show()


Notes on bureau_1_b:
* Not all `case_id` exist in bureau_1_b. Presumably the remaining are in bureau_1_a
* Some `case_id` have multiple rows. Presumably the borrower made multiple loans. However, some of these loans have no amount
* Potential features to be used:
    * dpdmax_851P	
    * maxdebtpduevalodued_3940955A	
    * overdueamountmax_950A	
    * debtpastduevalue_732A	
    * pmtdaysoverdue_1135P

In [6]:
combined_df = base_df.merge(bureau_b_1, on=["case_id"])
combined_df.head(5)

,case_id,date_decision,MONTH,WEEK_NUM,target,amount_1115A,classificationofcontr_1114M,contractdate_551D,contractmaturitydate_151D,contractst_516M,...,pmtnumpending_403L,purposeofcred_722M,residualamount_1093A,residualamount_127A,residualamount_3940956A,subjectrole_326M,subjectrole_43M,totalamount_503A,totalamount_881A,contract_length_year
0,467,2019-01-08,201901,1,0,NaN,ea6782cc,2011-06-15,2031-06-13,7241344e,...,NaN,96a8fdfe,0.0,0.0,NaN,fa4f56f1,ab3c25cf,3000000.00,10000.0,20.008219
1,467,2019-01-08,201901,1,0,NaN,ea6782cc,2019-01-04,2021-08-04,7241344e,...,NaN,96a8fdfe,NaN,NaN,NaN,ab3c25cf,ab3c25cf,78000.00,960000.0,2.583562
2,467,2019-01-08,201901,1,0,78000.0,ea6782cc,2016-10-25,2019-10-25,7241344e,...,10.0,96a8fdfe,NaN,NaN,NaN,a55475b1,a55475b1,NaN,NaN,3.000000
3,1445,2019-01-16,201901,2,0,NaN,ea6782cc,2015-01-30,2021-01-30,7241344e,...,NaN,60c73645,0.0,0.0,73044.18,daf49a8a,ab3c25cf,400000.00,100000.0,6.005479
4,1445,2019-01-16,201901,2,0,NaN,01f63ac8,2014-09-12,2021-09-12,7241344e,...,NaN,96a8fdfe,NaN,NaN,NaN,ab3c25cf,ab3c25cf,396800.62,184587.8,7.005479


In [41]:
combined_true = combined_df[combined_df["target"]==1].reset_index(drop=True)
combined_false = combined_df[combined_df["target"]==0].reset_index(drop=True)

combined_false.head(5)

,case_id,date_decision,MONTH,WEEK_NUM,target,amount_1115A,classificationofcontr_1114M,contractdate_551D,contractmaturitydate_151D,contractst_516M,...,pmtnumpending_403L,purposeofcred_722M,residualamount_1093A,residualamount_127A,residualamount_3940956A,subjectrole_326M,subjectrole_43M,totalamount_503A,totalamount_881A,contract_length_year
0,467,2019-01-08,201901,1,0,NaN,ea6782cc,2011-06-15,2031-06-13,7241344e,...,NaN,96a8fdfe,0.000,0.000,NaN,fa4f56f1,ab3c25cf,"3,000,000.000","10,000.000",20.008
1,467,2019-01-08,201901,1,0,NaN,ea6782cc,2019-01-04,2021-08-04,7241344e,...,NaN,96a8fdfe,NaN,NaN,NaN,ab3c25cf,ab3c25cf,"78,000.000","960,000.000",2.584
2,467,2019-01-08,201901,1,0,"78,000.000",ea6782cc,2016-10-25,2019-10-25,7241344e,...,10.000,96a8fdfe,NaN,NaN,NaN,a55475b1,a55475b1,NaN,NaN,3.000
3,1445,2019-01-16,201901,2,0,NaN,ea6782cc,2015-01-30,2021-01-30,7241344e,...,NaN,60c73645,0.000,0.000,"73,044.180",daf49a8a,ab3c25cf,"400,000.000","100,000.000",6.005
4,1445,2019-01-16,201901,2,0,NaN,01f63ac8,2014-09-12,2021-09-12,7241344e,...,NaN,96a8fdfe,NaN,NaN,NaN,ab3c25cf,ab3c25cf,"396,800.620","184,587.800",7.005


In [48]:
temp = stats.ttest_ind(combined_true["contract_length_year"].dropna(), combined_false["contract_length_year"].dropna())

In [115]:
def stats_tests(dataframe: pd.DataFrame):

    cols = [val for val in dataframe.columns if val!="target"]
    pvals = []
    tstats = []
    

    true_df = dataframe[dataframe["target"]==1]
    false_df = dataframe[dataframe["target"]==0]

    for col in cols:

        if is_numeric_dtype(dataframe[col]):
            result = stats.ttest_ind(true_df[col], false_df[col], nan_policy="omit")

            tstats.append(result.statistic)
            pvals.append(result.pvalue)

    result_df = pd.DataFrame(list(zip(cols, tstats, pvals)), columns=["col_name", "tstat", "pvalue"])

    return result_df
        

In [61]:
temp.pvalue

1.2449645529642165e-40

In [117]:
nan_10 = bureau_b_1_summary.loc[bureau_b_1_summary["nan_percent"]<10, "column_name"].to_list()
nan_10.append("target")

pval_df = stats_tests(combined_df[nan_10])

pval_df


,col_name,tstat,pvalue
0,case_id,-5.110,0.000
1,num_group1,8.836,0.000
2,contract_length_year,-13.354,0.000
3,dpdmax_851P,2.081,0.037
4,dpdmaxdatemonth_804T,-0.839,0.401
5,dpdmaxdateyear_742T,18.781,0.000
6,maxdebtpduevalodued_3940955A,2.285,0.022
7,overdueamountmax_950A,2.607,0.009
8,overdueamountmaxdatemonth_494T,-0.316,0.752
9,overdueamountmaxdateyear_432T,26.062,0.000


In [151]:
sig_numeric = pval_df.loc[pval_df["pvalue"]<0.05, "col_name"].to_list()
sig_numeric = [val for ind, val in enumerate(sig_numeric) if ind>1]
sig_numeric.append("target")

In [120]:
combined_df[sig_numeric]

,contract_length_year,dpdmax_851P,dpdmaxdateyear_742T,maxdebtpduevalodued_3940955A,overdueamountmax_950A,overdueamountmaxdateyear_432T,pmtdaysoverdue_1135P
0,20.008,NaN,NaN,NaN,NaN,NaN,NaN
1,2.584,NaN,NaN,NaN,NaN,NaN,NaN
2,3.000,0.000,"2,016.000",0.000,0.000,"2,016.000",0.000
3,6.005,"200,418.000","2,018.000",0.400,1.400,"2,018.000",0.000
4,7.005,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
85786,2.003,"38,126.000","2,019.000",0.200,0.400,"2,020.000",0.000
85787,5.003,"185,729.000","2,020.000",8.600,10.400,"2,020.000",0.000
85788,8.005,"15,304.000","2,015.000",0.200,1.000,"2,015.000",0.000
85789,4.003,"128,425.000","2,018.000",0.400,5.200,"2,018.000",0.000


In [113]:
pval_df.dtypes

col_name     object
tstat       float64
pvalue      float64
dtype: object

----
----
# <center> Model </center>
----

In [154]:
import math

import plotly.graph_objects as go
import tensorflow as tf


from keras import Model, Input
from keras.layers import Dense, Dropout
from keras.losses import BinaryCrossentropy
from keras.optimizers import Adam
from keras.metrics import Accuracy, AUC, TruePositives, TrueNegatives, FalsePositives, FalseNegatives
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split


In [98]:
combined_df[sig_numeric].dropna()

,contract_length_year,dpdmax_851P,dpdmaxdateyear_742T,maxdebtpduevalodued_3940955A,overdueamountmax_950A,overdueamountmaxdateyear_432T,pmtdaysoverdue_1135P,contractdate_551D,contractmaturitydate_151D,classificationofcontr_1114M,contractst_516M,contracttype_653M,credor_3940957M,pmtmethod_731M,purposeofcred_722M,subjectrole_326M,subjectrole_43M,periodicityofpmts_997M,lastupdate_260D
2,3.000,0.000,"2,016.000",0.000,0.000,"2,016.000",0.000,2016-10-25,2019-10-25,ea6782cc,7241344e,4257cbed,c5a72b57,e914c86c,96a8fdfe,a55475b1,a55475b1,a0b598e4,2019-01-10
3,6.005,"200,418.000","2,018.000",0.400,1.400,"2,018.000",0.000,2015-01-30,2021-01-30,ea6782cc,7241344e,1c9c5356,b619fa46,a55475b1,60c73645,daf49a8a,ab3c25cf,a55475b1,2019-01-19
5,0.079,0.000,"2,019.000",0.000,0.000,"2,019.000",0.000,2018-12-31,2019-01-29,ea6782cc,7241344e,4257cbed,0aebc0bb,dbcbe8f8,96a8fdfe,a55475b1,a55475b1,d479a207,2019-01-27
6,1.419,0.000,"2,018.000",0.000,0.000,"2,018.000",0.000,2018-07-25,2019-12-25,01f63ac8,7241344e,4257cbed,50babcd4,dbcbe8f8,60c73645,a55475b1,a55475b1,a0b598e4,2019-01-28
7,7.005,"49,765.000","2,017.000",5.800,7.200,"2,017.000",0.000,2014-11-27,2021-11-27,01f63ac8,7241344e,4257cbed,74bd67a8,e914c86c,60c73645,a55475b1,a55475b1,a0b598e4,2019-01-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82514,2.003,0.000,"2,019.000",0.000,0.000,"2,019.000",0.000,2019-12-06,2021-12-06,ea6782cc,7241344e,1c9c5356,b619fa46,a55475b1,60c73645,ab3c25cf,ab3c25cf,a55475b1,2020-10-10
82515,1.000,0.000,"2,020.000",0.000,0.000,"2,020.000",0.000,2020-09-02,2021-09-02,ea6782cc,7241344e,4257cbed,P0_31_66,f6e26148,60c73645,a55475b1,a55475b1,a0b598e4,2020-10-13
82516,2.000,0.000,"2,020.000",0.000,0.000,"2,020.000",0.000,2020-03-16,2022-03-16,ea6782cc,7241344e,1c9c5356,b619fa46,a55475b1,60c73645,ab3c25cf,ab3c25cf,a55475b1,2020-10-02
82517,0.748,0.000,"2,020.000",0.000,0.000,"2,020.000",0.000,2020-06-29,2021-03-29,ea6782cc,7241344e,1c9c5356,c5a72b57,e914c86c,96a8fdfe,ab3c25cf,ab3c25cf,a55475b1,2020-10-14


In [189]:
def class_imbalance_treatment(df: pd.DataFrame):

    true_class = df[df["target"]==1]
    false_class = df[df["target"]==0]

    # true_to_false = len(true_class)/len(false_class)
    duplicate_factor = math.floor(len(false_class)/len(true_class))

    ##### create duplicated rows to increase the number of true class
    duplicated_true_class = true_class.loc[true_class.index.repeat(duplicate_factor)]

    ##### concatenate, reshuffle, and reindex dataframe
    new_df = pd.concat([false_class, duplicated_true_class], ignore_index=True)
    new_df = new_df.sample(frac=1)
    new_df = new_df.reset_index(drop=True)

    ##### random sampling of new_df to reduce size
    new_df_len = len(new_df)

    rng = np.random.default_rng(43)
    random_sample = rng.integers(low=0, high=new_df_len, size=math.floor(new_df_len/2))

    new_df = new_df.loc[random_sample, :]

    return new_df
    

    


In [196]:
# feature_names = ["contract_length_year", "dpdmax_851P", "dpdmaxdatemonth_804T", "dpdmaxdateyear_742T", "overdueamountmax_950A", "target"]

# sig_numeric.append("target")
feature_names = sig_numeric

numeric_features = combined_df[feature_names].dropna()
numeric_features = class_imbalance_treatment(numeric_features)

target = numeric_features["target"]

numeric_features = numeric_features.drop(columns=["target"])

numeric_features

,contract_length_year,dpdmax_851P,dpdmaxdateyear_742T,maxdebtpduevalodued_3940955A,overdueamountmax_950A,overdueamountmaxdateyear_432T,pmtdaysoverdue_1135P
75375,2.003,"9,203.000","2,019.000",1.200,1.200,"2,019.000",0.000
97298,8.005,"432,076.000","2,016.000",180.200,235.200,"2,020.000","1,058.000"
59860,0.058,"27,412.000","2,017.000",118.000,118.000,"2,019.000",531.000
6529,4.003,"102,582.000","2,018.000",122.000,143.200,"2,019.000",632.000
86089,0.055,0.000,"2,019.000",0.000,0.000,"2,019.000",0.000
...,...,...,...,...,...,...,...
45212,2.003,0.000,"2,014.000",0.000,0.000,"2,014.000",0.000
84764,2.003,0.000,"2,019.000",0.000,0.000,"2,019.000",0.000
122981,6.005,0.000,"2,018.000",0.000,0.000,"2,018.000",0.000
142844,1.501,0.000,"2,019.000",0.000,0.000,"2,019.000",0.000


In [197]:
feature_train, feature_test, target_train, target_test = train_test_split(numeric_features, target, test_size=0.3)
feature_train

,contract_length_year,dpdmax_851P,dpdmaxdateyear_742T,maxdebtpduevalodued_3940955A,overdueamountmax_950A,overdueamountmaxdateyear_432T,pmtdaysoverdue_1135P
19597,0.082,0.000,"2,020.000",0.000,0.000,"2,020.000",0.000
107304,2.252,"24,845.000","2,020.000",0.800,0.800,"2,020.000",0.000
86652,2.003,"137,962.000","2,019.000",16.800,33.800,"2,020.000",152.000
127816,3.041,0.000,"2,020.000",0.000,0.000,"2,020.000",0.000
29860,2.003,0.000,"2,019.000",0.000,0.000,"2,019.000",0.000
...,...,...,...,...,...,...,...
30144,8.005,0.000,"2,013.000",0.000,0.000,"2,013.000",0.000
144285,1.507,0.000,"2,019.000",0.000,0.000,"2,019.000",0.000
40696,2.000,"22,907.000","2,019.000",2.800,2.800,"2,019.000",0.000
93764,1.332,"19,006.000","2,020.000",1.200,1.200,"2,020.000",0.000


In [198]:
feature_train.shape

(52206, 7)

In [199]:
##### Model definition

### input layer
input_layer = Input(shape=(7, ))
mid_layer = Dense(7, activation="relu")(input_layer)
mid_layer = Dense(9,activation="relu")(mid_layer)
mid_layer = Dense(7,activation="relu")(mid_layer)
mid_layer = Dense(5,activation="relu")(mid_layer)
mid_layer = Dense(3,activation="relu")(mid_layer)
output_layer = Dense(1, activation="sigmoid")(mid_layer)

### define model
model = Model(inputs=input_layer, outputs=output_layer)

In [200]:
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 7)]               0         
                                                                 
 dense_22 (Dense)            (None, 7)                 56        
                                                                 
 dense_23 (Dense)            (None, 9)                 72        
                                                                 
 dense_24 (Dense)            (None, 7)                 70        
                                                                 
 dense_25 (Dense)            (None, 5)                 40        
                                                                 
 dense_26 (Dense)            (None, 3)                 18        
                                                                 
 dense_27 (Dense)            (None, 1)                 4   

In [218]:
model.compile(
    loss=BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.legacy.Adam(
        learning_rate=0.0001,
    ),
    metrics=[AUC(name="auc_roc")]
)

In [219]:
result = model.fit(feature_train, 
                   target_train,
                   epochs=100,
                   validation_split=0.1)

Epoch 1/100


1469/1469 [==============================] - 1s 455us/step - loss: 0.6978 - auc_roc: 0.4994 - val_loss: 0.6963 - val_auc_roc: 0.5000
Epoch 2/100
1469/1469 [==============================] - 1s 386us/step - loss: 0.6954 - auc_roc: 0.5037 - val_loss: 0.6946 - val_auc_roc: 0.5000
Epoch 3/100
1469/1469 [==============================] - 1s 402us/step - loss: 0.6941 - auc_roc: 0.5019 - val_loss: 0.6937 - val_auc_roc: 0.5000
Epoch 4/100
1469/1469 [==============================] - 1s 381us/step - loss: 0.6935 - auc_roc: 0.5033 - val_loss: 0.6934 - val_auc_roc: 0.5000
Epoch 5/100
1469/1469 [==============================] - 1s 384us/step - loss: 0.6933 - auc_roc: 0.5048 - val_loss: 0.6932 - val_auc_roc: 0.5000
Epoch 6/100
1469/1469 [==============================] - 1s 381us/step - loss: 0.6932 - auc_roc: 0.4963 - val_loss: 0.6931 - val_auc_roc: 0.5000
Epoch 7/100
1469/1469 [==============================] - 1s 382us/step - loss: 0.6931 - auc_roc: 0.5000 - val_loss: 0.6931 - val_auc_roc: 0.50

In [147]:
def plot_fit_result(keras_object):

    epochs = keras_object.epoch

    train_loss = keras_object.history["loss"]
    train_auc = keras_object.history["auc_roc"]
    validation_loss = keras_object.history["val_loss"]
    validation_auc = keras_object.history["val_auc_roc"]

    fig = make_subplots(rows=1, cols=2)

    fig.add_trace(go.Scatter(x=epochs, y=train_loss, name="train loss"), row=1, col=1)
    fig.add_trace(go.Scatter(x=epochs, y=train_auc, name="train auc"), row=1, col=2)

    fig.add_trace(go.Scatter(x=epochs, y=validation_loss, name="validation loss"), row=1, col=1)
    fig.add_trace(go.Scatter(x=epochs, y=validation_auc, name="validation auc"), row=1, col=2)

    fig.show()



In [221]:
plot_fit_result(result)

In [213]:
result.history.keys()

dict_keys(['loss', 'auc_roc', 'val_loss', 'val_auc_roc'])

In [205]:
test_scores = model.evaluate(feature_test, target_test)

print(f"Test loss: {test_scores[0]}")
print(f"Test AUC: {test_scores[1]}")

102/700 [===>..........................] - ETA: 0s - loss: 0.6930 - auc_roc: 0.5000

700/700 [==============================] - 0s 321us/step - loss: 0.6930 - auc_roc: 0.5000
Test loss: 0.6930053234100342
Test AUC: 0.5


In [60]:
result = model.predict(feature_test)


760/760 [==============================] - 0s 203us/step


In [142]:
test_scores

[0.23903490602970123, 0.5]

In [143]:
model.metrics_names

['loss', 'auc_3']

---
---
# <center> Old Stuff </center>
---

In [178]:
person_df_1 = pd.read_csv("./csv_files/train/train_person_1.csv")
person_df_1.head(5)


,case_id,birth_259D,birthdate_87D,childnum_185L,contaddr_district_15M,contaddr_matchlist_1032L,contaddr_smempladdr_334L,contaddr_zipcode_807M,education_927M,empl_employedfrom_271D,...,registaddr_district_1083M,registaddr_zipcode_184M,relationshiptoclient_415T,relationshiptoclient_642T,remitter_829L,role_1084L,role_993L,safeguarantyflag_411L,sex_738L,type_25L
0,0,1986-07-01,NaN,NaN,P88_18_84,False,False,P167_100_165,P97_36_170,2017-09-15,...,P88_18_84,P167_100_165,NaN,NaN,NaN,CL,NaN,True,F,PRIMARY_MOBILE
1,0,NaN,NaN,NaN,a55475b1,NaN,NaN,a55475b1,a55475b1,NaN,...,a55475b1,a55475b1,SPOUSE,NaN,False,EM,NaN,NaN,NaN,PHONE
2,0,NaN,NaN,NaN,a55475b1,NaN,NaN,a55475b1,a55475b1,NaN,...,a55475b1,a55475b1,COLLEAGUE,SPOUSE,False,PE,NaN,NaN,NaN,PHONE
3,0,NaN,NaN,NaN,a55475b1,NaN,NaN,a55475b1,a55475b1,NaN,...,a55475b1,a55475b1,NaN,COLLEAGUE,NaN,PE,NaN,NaN,NaN,PHONE
4,1,1957-08-01,NaN,NaN,P103_93_94,False,False,P176_37_166,P97_36_170,2008-10-29,...,P103_93_94,P176_37_166,NaN,NaN,NaN,CL,NaN,True,M,PRIMARY_MOBILE


In [217]:
person_df_1_summary = dataframe_summary(person_df_1, 15)
person_df_1_summary

,column_name,unique_count,nan_count,nan_percent,data_type,unique_entries
0,case_id,1526659,0.0,0.000000,int64,>15 entries
23,num_group1,10,0.0,0.000000,int64,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]"
25,persontype_1072L,4,6117.0,0.205683,float64,"[1.0, 4.0, 5.0, nan]"
24,personindex_1023L,8,642283.0,21.596669,float64,"[0.0, 1.0, 2.0, nan, 3.0, 4.0, 5.0, 6.0]"
26,persontype_792L,4,642283.0,21.596669,float64,"[1.0, 4.0, 5.0, nan]"
21,mainoccupationinc_384A,6633,1447332.0,48.666321,float64,>15 entries
3,childnum_185L,12,2964084.0,99.666879,float64,"[nan, 0.0, 1.0, 3.0, 2.0, 4.0, 7.0, 6.0, 5.0, ..."
4,contaddr_district_15M,975,0.0,0.000000,object,>15 entries
7,contaddr_zipcode_807M,3530,0.0,0.000000,object,>15 entries
8,education_927M,6,0.0,0.000000,object,"[P97_36_170, a55475b1, P33_146_175, P17_36_170..."


In [229]:
person_df_1_desc = describe_dataframe(person_df_1)
person_df_1_desc

,case_id,childnum_185L,mainoccupationinc_384A,num_group1,personindex_1023L,persontype_1072L,persontype_792L
count,"2,973,991.000","9,907.000","1,526,659.000","2,973,991.000","2,331,708.000","2,967,874.000","2,331,708.000"
mean,"1,055,195.612",0.616,"57,707.483",0.797,0.438,2.035,2.316
std,"724,571.385",0.966,"33,348.303",0.978,0.660,1.707,1.826
min,0.000,0.000,0.000,0.000,0.000,1.000,1.000
25%,"637,353.500",0.000,"36,000.000",0.000,0.000,1.000,1.000
50%,"890,817.000",0.000,"50,000.000",0.000,0.000,1.000,1.000
75%,"1,568,333.500",1.000,"70,000.000",1.000,1.000,4.000,5.000
max,"2,703,454.000",11.000,"200,000.000",9.000,6.000,5.000,5.000


In [242]:
columns_of_interest = ["case_id", "birth_259D", "birthdate_87D", "maritalst_703L", "familystate_447L",
                       "relationshiptoclient_415T", "relationshiptoclient_642T", "num_group1",
                       "persontype_1072L", "personindex_1023L", "persontype_792L"]
person_df_1.loc[person_df_1["case_id"].isin([31, 97, 123, 913, 0]), columns_of_interest]

,case_id,birth_259D,birthdate_87D,maritalst_703L,familystate_447L,relationshiptoclient_415T,relationshiptoclient_642T,num_group1,persontype_1072L,personindex_1023L,persontype_792L
0,0,1986-07-01,NaN,NaN,MARRIED,NaN,NaN,0,1.000,0.000,1.000
1,0,NaN,NaN,NaN,NaN,SPOUSE,NaN,1,1.000,1.000,4.000
2,0,NaN,NaN,NaN,NaN,COLLEAGUE,SPOUSE,2,4.000,2.000,5.000
3,0,NaN,NaN,NaN,NaN,NaN,COLLEAGUE,3,5.000,NaN,NaN
119,31,1972-07-01,NaN,NaN,MARRIED,NaN,NaN,0,1.000,0.000,1.000
120,31,NaN,NaN,NaN,NaN,CHILD,NaN,1,1.000,1.000,5.000
121,31,NaN,NaN,NaN,NaN,FRIEND,CHILD,2,5.000,2.000,5.000
122,31,NaN,NaN,NaN,NaN,NaN,FRIEND,3,5.000,NaN,NaN
356,97,1986-04-01,NaN,NaN,MARRIED,NaN,NaN,0,1.000,0.000,1.000
357,97,NaN,NaN,NaN,NaN,SPOUSE,NaN,1,1.000,1.000,4.000


Notes on person_df_1:
* Basically each `case_id` represents individual cases with `num_group1` == 0, that is the applicants. Other values of `num_group1` with the same `case_id` are different individuals related to the applicants.
* `relationshiptoclient_415T` and `relationshiptoclient_642T` are 2 different descriptions but not too sure what
* potential features to be use:
    * birth_259D -> get age in 2024
    * mainoccupationinc_384A
    * education_927M
    * contaddr_district_15M
    * contaddr_zipcode_807M
    * incometype_1044T
    * sex_738L
    * language1_981M


In [247]:
person_df_2 = pd.read_csv("./csv_files/train/train_person_2.csv")
person_df_2.head(5)

,case_id,addres_district_368M,addres_role_871L,addres_zip_823M,conts_role_79M,empls_economicalst_849M,empls_employedfrom_796D,empls_employer_name_740M,num_group1,num_group2,relatedpersons_role_762T
0,5,a55475b1,NaN,a55475b1,a55475b1,a55475b1,NaN,a55475b1,0,0,NaN
1,6,P55_110_32,CONTACT,P10_68_40,P38_92_157,P164_110_33,NaN,a55475b1,0,0,NaN
2,6,P55_110_32,PERMANENT,P10_68_40,a55475b1,a55475b1,NaN,a55475b1,0,1,NaN
3,6,P204_92_178,CONTACT,P65_136_169,P38_92_157,P164_110_33,NaN,a55475b1,1,0,OTHER_RELATIVE
4,6,P191_109_75,CONTACT,P10_68_40,P7_147_157,a55475b1,NaN,a55475b1,1,1,OTHER_RELATIVE


In [248]:
person_df_2_summary = dataframe_summary(person_df_2, 15)
person_df_2_summary

,column_name,unique_count,nan_count,nan_percent,data_type,unique_entries
0,case_id,1435105,0.000,0.000,int64,>15 entries
8,num_group1,5,0.000,0.000,int64,"[0, 1, 2, 3, 4]"
9,num_group2,32,0.000,0.000,int64,>15 entries
1,addres_district_368M,508,0.000,0.000,object,>15 entries
3,addres_zip_823M,2027,0.000,0.000,object,>15 entries
4,conts_role_79M,11,0.000,0.000,object,"[a55475b1, P38_92_157, P7_147_157, P177_137_98..."
5,empls_economicalst_849M,10,0.000,0.000,object,"[a55475b1, P164_110_33, P22_131_138, P28_32_17..."
7,empls_employer_name_740M,7153,0.000,0.000,object,>15 entries
2,addres_role_871L,9,"1,575,736.000",95.882,object,"[nan, CONTACT, PERMANENT, TEMPORARY, REGISTERE..."
10,relatedpersons_role_762T,11,"1,614,684.000",98.252,object,"[nan, OTHER_RELATIVE, PARENT, COLLEAGUE, FRIEN..."


Notes on person_df_2:
* There are some intersection between `case_id` in person_df_1 and person_df_2. However, some `case_id` may only exist in either one of the tables
* Potential features to be used:
    * addres_district_368M
    * addres_zip_823M	
    * conts_role_79M

In [310]:
static_df_1 = pd.read_csv("./csv_files/train/train_static_0_0.csv")
static_df_1



,case_id,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,...,sellerplacecnt_915L,sellerplacescnt_216L,sumoutstandtotal_3546847A,sumoutstandtotalest_4493215A,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A,twobodfilling_608L,typesuite_864L,validfrom_1069D
0,0,NaN,NaN,"1,917.600",0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,NaN,NaN,0.000,0.000,NaN,BO,NaN,NaN
1,1,NaN,NaN,"3,134.000",0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,NaN,NaN,0.000,0.000,NaN,BO,NaN,NaN
2,2,NaN,NaN,"4,937.000",0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,NaN,NaN,0.000,0.000,NaN,BO,AL,NaN
3,3,NaN,NaN,"4,643.600",0.000,0.000,1.000,0.000,2.000,0.000,...,1.000,1.000,NaN,NaN,0.000,0.000,NaN,BO,AL,NaN
4,4,NaN,NaN,"3,390.200",0.000,0.000,1.000,0.000,0.000,0.000,...,0.000,0.000,NaN,NaN,0.000,0.000,NaN,BO,AL,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003752,2651088,0.000,"117,624.790","3,045.000","4,488.000",0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,"44,848.000","44,848.000","44,848.000","199,664.670","8,976.000",FO,NaN,2019-11-12
1003753,2651089,0.000,"134,887.400","1,200.000","4,382.800",0.000,0.000,0.000,0.000,0.000,...,0.000,10.000,"39,392.000","39,392.000","39,392.000","209,840.000","8,765.601",FO,NaN,2019-11-12
1003754,2651090,0.000,"69,186.620","6,000.000",0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,"76,455.620",NaN,FO,NaN,2019-11-12
1003755,2651091,0.000,"117,331.000","11,565.400","13,781.200",0.000,0.000,0.000,0.000,0.000,...,0.000,2.000,"55,084.402","55,084.402","55,084.402","168,248.310","13,781.200",FO,NaN,2019-11-30


In [311]:
static_summary_1 = dataframe_summary(static_df_1, 15)
static_summary_1

,column_name,unique_count,nan_count,nan_percent,data_type,unique_entries
64,isbidproduct_1095L,2,0.000,0.000,bool,"[False, True]"
0,case_id,1003757,0.000,0.000,int64,>15 entries
3,annuity_780A,67308,0.000,0.000,float64,>15 entries
5,applicationcnt_361L,5,0.000,0.000,float64,"[0.0, 2.0, 1.0, 3.0, 5.0]"
6,applications30d_658L,21,0.000,0.000,float64,>15 entries
...,...,...,...,...,...,...
56,equalityempfrom_62L,3,"975,352.000",97.170,object,"[nan, True, False]"
65,isbidproductrequest_292L,3,"995,281.000",99.156,object,"[nan, False, True]"
84,lastrepayingdate_696D,332,"1,002,152.000",99.840,object,>15 entries
146,payvacationpostpone_4187118D,311,"1,002,290.000",99.854,object,>15 entries


In [337]:
zero_nan = static_summary_1.loc[(static_summary_1["nan_percent"]==0
                                 & static_summary_1["data_type"].isin(["int64", "float64"])), "column_name"].to_list()
zero_nan


sub_static_1 = static_df_1[zero_nan].select_dtypes([int, float])
# sub_static_1 = sub_static_1[]

sub_static_1


,case_id,annuity_780A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,applicationscnt_867L,clientscnt12m_3712952L,clientscnt3m_3712950L,...,mobilephncnt_593L,numactivecreds_622L,numactivecredschannel_414L,numactiverelcontr_750L,numcontrs3months_479L,numnotactivated_1143L,numpmtchanneldd_318L,numrejects9m_859L,sellerplacecnt_915L,sellerplacescnt_216L
0,0,"1,917.600",0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,1,"3,134.000",0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,2,"4,937.000",0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,2.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,3,"4,643.600",0.000,1.000,0.000,2.000,0.000,1.000,0.000,0.000,...,1.000,0.000,0.000,0.000,1.000,0.000,0.000,1.000,1.000,1.000
4,4,"3,390.200",0.000,1.000,0.000,0.000,0.000,1.000,0.000,0.000,...,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003752,2651088,"3,045.000",0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,3.000,1.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000
1003753,2651089,"1,200.000",0.000,0.000,0.000,0.000,0.000,11.000,0.000,0.000,...,2.000,1.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,10.000
1003754,2651090,"6,000.000",0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,2.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000
1003755,2651091,"11,565.400",0.000,0.000,0.000,0.000,0.000,2.000,0.000,0.000,...,2.000,1.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,2.000


In [343]:
sub_static_1_desc = describe_dataframe(sub_static_1).T
sub_static_1_desc["skewdness"] = (3*sub_static_1_desc["mean"]-sub_static_1_desc["50%"])/sub_static_1_desc["std"]
sub_static_1_desc

,count,mean,std,min,25%,50%,75%,max,skewdness
case_id,"1,003,757.000","1,216,924.056","696,462.442",0.000,"725,987.000","1,311,700.000","1,562,639.000","2,651,092.000",3.359
annuity_780A,"1,003,757.000","3,875.598","2,920.237",83.000,"1,895.800","3,000.000","4,989.800","106,007.000",2.954
applicationcnt_361L,"1,003,757.000",0.000,0.009,0.000,0.000,0.000,0.000,5.000,0.009
applications30d_658L,"1,003,757.000",0.126,0.446,0.000,0.000,0.000,0.000,25.000,0.849
applicationscnt_1086L,"1,003,757.000",0.420,2.683,0.000,0.000,0.000,0.000,443.000,0.470
applicationscnt_464L,"1,003,757.000",1.239,10.316,0.000,0.000,0.000,0.000,247.000,0.360
applicationscnt_629L,"1,003,757.000",0.316,2.648,0.000,0.000,0.000,0.000,77.000,0.358
applicationscnt_867L,"1,003,757.000",2.547,3.742,0.000,0.000,1.000,3.000,97.000,1.775
clientscnt12m_3712952L,"1,003,757.000",0.032,0.277,0.000,0.000,0.000,0.000,47.000,0.346
clientscnt3m_3712950L,"1,003,757.000",0.015,0.228,0.000,0.000,0.000,0.000,47.000,0.195


In [318]:
static_df_1.loc[static_df_1["clientscnt12m_3712952L"]>0, ["case_id", "annuity_780A", "applicationcnt_361L", "clientscnt12m_3712952L"]]
# static_df_1[static_df_1["clientscnt12m_3712952L"]>0]

,case_id,annuity_780A,applicationcnt_361L,clientscnt12m_3712952L
20,20,"6,000.000",0.000,1.000
34,34,"3,654.000",0.000,1.000
43,43,"2,733.200",0.000,1.000
44,44,"2,555.000",0.000,1.000
149,149,"4,196.600",0.000,1.000
...,...,...,...,...
1003645,2650961,"6,117.000",0.000,1.000
1003656,2650973,"6,302.000",0.000,1.000
1003697,2651017,"12,591.200",0.000,1.000
1003714,2651038,"2,016.000",0.000,1.000


----
----
# <center> Experimentation </center>
----

In [172]:
def dataframe_summary(dataframe: pd.DataFrame):

    ### Get list of column names
    col_names = dataframe.columns.to_list()

    ### Get unique count of each column
    col_length = [len(dataframe[val].unique()) for val in col_names]

    ### Append info for dataframe
    col_names.append("this_df")
    col_length.append(len(dataframe))

    ### Create new dataframe for summary
    summary_df = pd.DataFrame(list(zip(col_names, col_length)), columns=["column_name", "unique_count"])

    ### Count nan values in each column
    nan_df = dataframe.isnull().sum(axis = 0).to_frame().reset_index().rename(columns={"index":"column_name", 0:"nan_count"})

    ### Create new frame for data type in each column
    type_df = dataframe.dtypes.to_frame().reset_index().rename(columns={"index":"column_name", 0:"data_type"})

    ### Merge summary_df with nan_df, type_df
    summary_df = summary_df.merge(nan_df, on="column_name", how="outer")
    summary_df = summary_df.merge(type_df, on="column_name", how="outer")

    return summary_df
    



In [173]:
temp = column_summary(person_df)
temp

,column_name,unique_count,nan_count,data_type
0,case_id,1526659,0.0,int64
1,birth_259D,681,1447332.0,object
2,birthdate_87D,660,2949075.0,object
3,childnum_185L,12,2964084.0,float64
4,contaddr_district_15M,975,0.0,object
5,contaddr_matchlist_1032L,2,1447773.0,object
6,contaddr_smempladdr_334L,3,1447773.0,object
7,contaddr_zipcode_807M,3530,0.0,object
8,education_927M,6,0.0,object
9,empl_employedfrom_271D,8076,2407290.0,object


In [164]:
temp = base_df.dtypes.to_frame().reset_index().rename(columns={"index":"column_name", 0:"data_type"})
temp
temp

,column_name,data_type
0,case_id,int64
1,date_decision,object
2,MONTH,int64
3,WEEK_NUM,int64
4,target,int64


In [141]:
sub_credit_df.groupby(["sex_738L", "target"]).count()

incometype_1044T  mainoccupationinc_384A  \
sex_738L target                                             
F        0                 927562                  927562   
         1                  25214                   25214   
M        0                 551103                  551103   
         1                  22780                   22780   

                 registaddr_zipcode_184M  birth_259D  applicant_bday  \
sex_738L target                                                        
F        0                        927562      927562          927562   
         1                         25214       25214           25214   
M        0                        551103      551103          551103   
         1                         22780       22780           22780   

                 age_in_2024  
sex_738L target               
F        0            927562  
         1             25214  
M        0            551103  
         1             22780

In [125]:
credit_df.dtypes

case_id                        int64
date_decision                 object
MONTH                          int64
WEEK_NUM                       int64
target                         int64
birth_259D                    object
contaddr_district_15M         object
contaddr_zipcode_807M         object
education_927M                object
empladdr_district_926M        object
empladdr_zipcode_114M         object
incometype_1044T              object
language1_981M                object
mainoccupationinc_384A       float64
num_group1                     int64
personindex_1023L            float64
persontype_1072L             float64
persontype_792L              float64
registaddr_district_1083M     object
registaddr_zipcode_184M       object
role_1084L                    object
safeguarantyflag_411L         object
sex_738L                      object
type_25L                      object
dtype: object

In [86]:
person_df_1[person_df_1["case_id"]==147982][["case_id", "num_group1", "personindex_1023L", "persontype_1072L",
                                             "", "persontype_792L", "role_1084L", "role_993L"]]

,case_id,num_group1,personindex_1023L,persontype_1072L,persontype_792L,role_1084L,role_993L
331801,147982,0,0.0,1.0,1.0,CL,NaN
331802,147982,1,1.0,1.0,5.0,EM,NaN
331803,147982,2,2.0,1.0,5.0,EM,NaN
331804,147982,3,3.0,1.0,5.0,CL,NaN
331805,147982,4,NaN,5.0,NaN,PE,NaN
331806,147982,5,NaN,5.0,NaN,PE,NaN
331807,147982,6,NaN,5.0,NaN,PE,NaN
331808,147982,7,NaN,5.0,NaN,PE,NaN
331809,147982,8,NaN,5.0,NaN,PE,NaN
331810,147982,9,NaN,5.0,NaN,PE,NaN


In [88]:
person_df_1[person_df_1["case_id"]==0]

,case_id,birth_259D,birthdate_87D,childnum_185L,contaddr_district_15M,contaddr_matchlist_1032L,contaddr_smempladdr_334L,contaddr_zipcode_807M,education_927M,empl_employedfrom_271D,...,registaddr_district_1083M,registaddr_zipcode_184M,relationshiptoclient_415T,relationshiptoclient_642T,remitter_829L,role_1084L,role_993L,safeguarantyflag_411L,sex_738L,type_25L
0,0,1986-07-01,NaN,NaN,P88_18_84,False,False,P167_100_165,P97_36_170,2017-09-15,...,P88_18_84,P167_100_165,NaN,NaN,NaN,CL,NaN,True,F,PRIMARY_MOBILE
1,0,NaN,NaN,NaN,a55475b1,NaN,NaN,a55475b1,a55475b1,NaN,...,a55475b1,a55475b1,SPOUSE,NaN,False,EM,NaN,NaN,NaN,PHONE
2,0,NaN,NaN,NaN,a55475b1,NaN,NaN,a55475b1,a55475b1,NaN,...,a55475b1,a55475b1,COLLEAGUE,SPOUSE,False,PE,NaN,NaN,NaN,PHONE
3,0,NaN,NaN,NaN,a55475b1,NaN,NaN,a55475b1,a55475b1,NaN,...,a55475b1,a55475b1,NaN,COLLEAGUE,NaN,PE,NaN,NaN,NaN,PHONE


In [92]:
person_df_1[person_df_1["case_id"]==0][["case_id", "num_group1", "personindex_1023L", "persontype_1072L", "maritalst_703L",
                                        "relationshiptoclient_415T", "relationshiptoclient_642T", "persontype_792L", 
                                        "role_1084L", "role_993L"]]

,case_id,num_group1,personindex_1023L,persontype_1072L,maritalst_703L,relationshiptoclient_415T,relationshiptoclient_642T,persontype_792L,role_1084L,role_993L
0,0,0,0.0,1.0,NaN,NaN,NaN,1.0,CL,NaN
1,0,1,1.0,1.0,NaN,SPOUSE,NaN,4.0,EM,NaN
2,0,2,2.0,4.0,NaN,COLLEAGUE,SPOUSE,5.0,PE,NaN
3,0,3,NaN,5.0,NaN,NaN,COLLEAGUE,NaN,PE,NaN


In [95]:
temp = person_df_1[person_df_1["num_group1"]==0]

In [96]:
temp

,case_id,birth_259D,birthdate_87D,childnum_185L,contaddr_district_15M,contaddr_matchlist_1032L,contaddr_smempladdr_334L,contaddr_zipcode_807M,education_927M,empl_employedfrom_271D,...,registaddr_district_1083M,registaddr_zipcode_184M,relationshiptoclient_415T,relationshiptoclient_642T,remitter_829L,role_1084L,role_993L,safeguarantyflag_411L,sex_738L,type_25L
0,0,1986-07-01,NaN,NaN,P88_18_84,False,False,P167_100_165,P97_36_170,2017-09-15,...,P88_18_84,P167_100_165,NaN,NaN,NaN,CL,NaN,True,F,PRIMARY_MOBILE
4,1,1957-08-01,NaN,NaN,P103_93_94,False,False,P176_37_166,P97_36_170,2008-10-29,...,P103_93_94,P176_37_166,NaN,NaN,NaN,CL,NaN,True,M,PRIMARY_MOBILE
9,2,1974-12-01,NaN,NaN,P91_125_184,False,False,P161_5_97,P97_36_170,2010-02-15,...,P91_125_184,P161_5_97,NaN,NaN,NaN,EM,NaN,True,F,PRIMARY_MOBILE
14,3,1993-08-01,NaN,NaN,P155_139_77,False,False,P161_14_174,P33_146_175,2018-05-15,...,P155_139_77,P161_14_174,NaN,NaN,NaN,CL,NaN,True,F,PRIMARY_MOBILE
17,4,1994-01-01,NaN,NaN,P194_82_174,False,False,P8_88_79,P33_146_175,2014-12-15,...,P194_82_174,P8_88_79,NaN,NaN,NaN,CL,NaN,True,F,PRIMARY_MOBILE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2973984,2703450,1960-01-01,NaN,NaN,P123_39_170,False,False,P40_33_175,a55475b1,NaN,...,P123_39_170,P40_33_175,NaN,NaN,NaN,CL,NaN,True,F,PRIMARY_MOBILE
2973985,2703451,1950-11-01,NaN,NaN,P162_18_172,False,False,P80_161_153,a55475b1,NaN,...,P162_18_172,P80_161_153,NaN,NaN,NaN,CL,NaN,True,F,PRIMARY_MOBILE
2973987,2703452,1977-08-01,NaN,NaN,P133_44_167,False,False,P59_150_74,a55475b1,NaN,...,P19_11_176,P11_15_81,NaN,NaN,NaN,CL,NaN,False,M,PRIMARY_MOBILE
2973988,2703453,1950-02-01,NaN,NaN,P123_6_84,False,False,P46_103_143,a55475b1,NaN,...,P123_6_84,P46_103_143,NaN,NaN,NaN,CL,NaN,False,F,PRIMARY_MOBILE


In [97]:
len(set(temp["case_id"].to_list()))

1526659

In [98]:
temp[temp["case_id"]==5]

,case_id,birth_259D,birthdate_87D,childnum_185L,contaddr_district_15M,contaddr_matchlist_1032L,contaddr_smempladdr_334L,contaddr_zipcode_807M,education_927M,empl_employedfrom_271D,...,registaddr_district_1083M,registaddr_zipcode_184M,relationshiptoclient_415T,relationshiptoclient_642T,remitter_829L,role_1084L,role_993L,safeguarantyflag_411L,sex_738L,type_25L
21,5,1979-10-01,NaN,NaN,P54_133_26,False,False,P66_157_171,P97_36_170,2016-01-15,...,P54_133_26,P66_157_171,NaN,NaN,NaN,CL,NaN,True,F,PRIMARY_MOBILE


In [33]:
person_df_2 = pd.read_csv("./csv_files/train/train_person_2.csv")
person_df_2

,case_id,addres_district_368M,addres_role_871L,addres_zip_823M,conts_role_79M,empls_economicalst_849M,empls_employedfrom_796D,empls_employer_name_740M,num_group1,num_group2,relatedpersons_role_762T
0,5,a55475b1,NaN,a55475b1,a55475b1,a55475b1,NaN,a55475b1,0,0,NaN
1,6,P55_110_32,CONTACT,P10_68_40,P38_92_157,P164_110_33,NaN,a55475b1,0,0,NaN
2,6,P55_110_32,PERMANENT,P10_68_40,a55475b1,a55475b1,NaN,a55475b1,0,1,NaN
3,6,P204_92_178,CONTACT,P65_136_169,P38_92_157,P164_110_33,NaN,a55475b1,1,0,OTHER_RELATIVE
4,6,P191_109_75,CONTACT,P10_68_40,P7_147_157,a55475b1,NaN,a55475b1,1,1,OTHER_RELATIVE
...,...,...,...,...,...,...,...,...,...,...,...
1643405,2703450,a55475b1,NaN,a55475b1,a55475b1,a55475b1,NaN,a55475b1,0,0,NaN
1643406,2703451,a55475b1,NaN,a55475b1,a55475b1,a55475b1,NaN,a55475b1,0,0,NaN
1643407,2703452,a55475b1,NaN,a55475b1,a55475b1,a55475b1,NaN,a55475b1,0,0,NaN
1643408,2703453,a55475b1,NaN,a55475b1,a55475b1,a55475b1,NaN,a55475b1,0,0,NaN


------


In [73]:
##### Code used to compile metadata

import os, warnings

warnings.filterwarnings("ignore")

path = "./csv_files/train/"
dir_list = os.listdir(path)

filename_dict = {}

for filename in dir_list:

    temp_df = pd.read_csv("./csv_files/train/" + filename)

    col_names = temp_df.columns.to_list()

    filename_dict[filename] = col_names

    print(filename)

files = []
columns = []
depth_list = []

for val in filename_dict.keys():

    files = files + [val]*len(filename_dict[val])
    columns = columns + filename_dict[val]
    print(val)



for val in files:

    temp = val[:-4].split("_")
    temp = [val for val in temp if val.isdigit()]

    if len(temp)>0:
        depth_list.append(int(temp[0]))
    else:
        depth_list.append(np.nan)


metadata = pd.DataFrame(list(zip(files, columns, depth_list)), columns=["filename", "col_names", "data_depth"])
metadata

train_static_0_1.csv
train_static_0_0.csv
train_static_cb_0.csv
train_other_1.csv
train_debitcard_1.csv
train_person_1.csv
train_person_2.csv
train_applprev_2.csv
train_credit_bureau_a_2_8.csv
train_credit_bureau_a_2_9.csv
train_credit_bureau_a_2_4.csv
train_credit_bureau_a_2_5.csv
train_tax_registry_b_1.csv
train_credit_bureau_a_2_10.csv
train_credit_bureau_a_2_7.csv
train_credit_bureau_a_2_6.csv
train_credit_bureau_a_2_2.csv
train_credit_bureau_a_2_3.csv
train_applprev_1_1.csv
train_deposit_1.csv
train_credit_bureau_a_2_1.csv
train_credit_bureau_a_2_0.csv
train_applprev_1_0.csv
train_tax_registry_a_1.csv
train_credit_bureau_b_1.csv
train_base.csv
train_credit_bureau_b_2.csv
train_tax_registry_c_1.csv
train_credit_bureau_a_1_3.csv
train_credit_bureau_a_1_2.csv
train_credit_bureau_a_1_0.csv
train_credit_bureau_a_1_1.csv
